In [1]:
import onnx
import yaml 
import os

device      = "cuda"
batch_size  = 64
config_path = "configs/sunset_configs.yaml"


with open(config_path, 'r') as file:
    configs = yaml.safe_load(file)

explorted_file = os.path.join(configs["model_dir"], "sunset_model.onnx")
onnx_model = onnx.load(explorted_file) 

onnx.checker.check_model(onnx_model) 
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input_image[FLOAT, batch_sizex48x64x64]
  %input_scalar[FLOAT, batch_sizex16]
) initializers (
  %conv1.weight[FLOAT, 48x48x3x3]
  %conv1.bias[FLOAT, 48]
  %batchnorm1.weight[FLOAT, 48]
  %batchnorm1.bias[FLOAT, 48]
  %batchnorm1.running_mean[FLOAT, 48]
  %batchnorm1.running_var[FLOAT, 48]
  %conv2.weight[FLOAT, 96x48x3x3]
  %conv2.bias[FLOAT, 96]
  %batchnorm2.weight[FLOAT, 96]
  %batchnorm2.bias[FLOAT, 96]
  %batchnorm2.running_mean[FLOAT, 96]
  %batchnorm2.running_var[FLOAT, 96]
  %concat.weight[FLOAT, 1024x24592]
  %concat.bias[FLOAT, 1024]
  %dense1.weight[FLOAT, 1024x1024]
  %dense1.bias[FLOAT, 1024]
  %dense2.weight[FLOAT, 15x1024]
  %dense2.bias[FLOAT, 15]
  %dense3.weight[FLOAT, 15x15]
  %dense3.bias[FLOAT, 15]
) {
  %/conv1/Conv_output_0 = Conv[dilations = [1, 1], group = 1, kernel_shape = [3, 3], pads = [1, 1, 1, 1], strides = [1, 1]](%input_image, %conv1.weight, %conv1.bias)
  %/relu/Relu_output_0 = Relu(%/conv1/Conv_output_0)
  %/batchnorm1/BatchNorma

In [3]:
import netron
# visualize the pre-trained model
netron.start(explorted_file)

Serving 'sunset_onnex_trained_torch/sunset_model.onnx' at http://localhost:8080


('localhost', 8080)

In [4]:
from dataloader import sirta_dataset

batch_size   = 8
test_dataset = sirta_dataset( mode = "Test",
                                irrad_path  = "sirta_data/2023",
                                image_path  = "sirta_data/2023/images",
                                seq_length  = 16,
                                pred_length = 15,
                                image_size  = 64,
                                batch_size  = batch_size,
                                training_index_file = "sirta_data/2023/training.txt",
                                validate_index_file = "sirta_data/2023/validate.txt",
                                testing_index_file  = "sirta_data/2023/testing.txt") 

/home/gaap/anaconda3/envs/onnx_sunset/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_validation.py:113: UserWarning: WARNING: failed to get cudart_version from onnxruntime build info.
  warnings.warn("WARNING: failed to get cudart_version from onnxruntime build info.")
/media/HDD1/Projects/CU/EE_SunshineX/onnx_sunset/myonnxutils/onnx_utils.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_csv_data["Datetime"] = pd.to_datetime(selected_csv_data["Datetime"])
image transformation: 100%|██████████| 14290/14290 [01:32<00:00, 154.64it/s]


Total number of stacked samples: 13360
For [Test] mode: the number of stacked samples: 668
               the number of batches: 83


In [5]:
import numpy as np
import tqdm
from myonnxutils.onnx_utils import mean_square_error
from torch.utils.data import Dataset, DataLoader
from onnxruntime import InferenceSession

session = InferenceSession(os.path.join(configs["model_dir"], "sunset_model.onnx"),providers=['CPUExecutionProvider'])

first_rmse_list = []
last_rmse_list  = [] 
batch_rmse_list = []

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True) 
pbar = tqdm.tqdm(test_loader)
for _, data_batch in enumerate(pbar): 

    input_index       = data_batch[0]
    input_iclr        = data_batch[1]
    input_skyimage    = data_batch[2]
    output_irr        = data_batch[3] 

    input_name1  = session.get_inputs()[0].name
    input_name2  = session.get_inputs()[1].name
    output_name  = session.get_outputs()[0].name 


    pred_irradiance_list_of_batches  = session.run(output_names=[output_name], input_feed={input_name1: input_skyimage.float().numpy(), input_name2: input_iclr.float().numpy() })

    
    first_rmse , last_rmse, batch_rmse = mean_square_error(np.concat(pred_irradiance_list_of_batches, axis=0), output_irr.float().numpy())

    first_rmse_list.append(first_rmse)
    last_rmse_list.append(last_rmse)
    batch_rmse_list.append(batch_rmse)
 
 
print(f'========== First RMSE: {sum(first_rmse_list)/len(first_rmse_list):.4f}, Last RMSE {sum(last_rmse_list)/len(last_rmse_list):.4f}, Batch RMSE {sum(batch_rmse_list)/len(batch_rmse_list):.4f}')

100%|██████████| 83/83 [00:03<00:00, 23.05it/s]

========== First RMSE: 56.9100, Last RMSE 73.0901, Batch RMSE 63.8340
